# Generating Zonal Statistics with a Custom Script
This notebook demonstrates how to generate zonal statistics using a custom Python script, with a focus on analyzing night light intensity across administrative regions in Rwanda. You will learn how to set up your workspace, import and prepare geospatial data, clip global raster datasets to a country boundary, and visualize the results. The workflow includes using custom utilities for zonal statistics, merging results with administrative boundaries, and saving the outputs for further analysis. This hands-on approach provides practical experience with geospatial data processing, raster analysis, and the integration of Python geospatial libraries such as geopandas and rasterio.

**Zonal statistics** are summary statistics (such as mean, sum, median, minimum, or maximum) calculated for values of a raster dataset within the boundaries of defined zones, such as polygons representing administrative regions or land cover types.


## Setup and Imports

### Required Packages

In [5]:
import sys
from pathlib import Path
import os
import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import warnings
warnings.filterwarnings('ignore')

In [30]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


### Workspace Directory Setup

In [31]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install rasterio

Note: you may need to restart the kernel to use updated packages.


## `Part 2: Virtual Environment Creation`

In [2]:
from osgeo import gdal
import geopandas, rasterio, pandas, numpy, matplotlib
print("All packages imported successfully!")
print(f"GDAL version: {gdal.__version__}")
print(f"GeoPandas version: {geopandas.__version__}")

All packages imported successfully!
GDAL version: 3.11.0
GeoPandas version: 1.1.1


## `Spatial Analysis`

In [6]:
DIR_WORKSPACE = Path.cwd().parents[0]
DIR_DATA = DIR_WORKSPACE / "nightlight-analysis/data"
DIR_SRC = DIR_WORKSPACE / "nightlight-analysis/src"

### Import Custom Python Script

In [7]:
# Add src directory to path to import our custom script
sys.path.append(str(DIR_SRC))


In [9]:
# We can now import our custom package
# There are cleaner ways to handle local imports but this works for now
from spatial_utils.zonal_stats import ZonalStatistics

### Input Data Files 
For the night lights file, please make sure you unzip it first. 

In [10]:
# ==========================================
# FILE PATHS
# ===========================================

# Unzipped night lights raster file with "tif" extension
FILE_NTL = DIR_DATA / "ntl/VNL_npp_2024_global_vcmslcfg_v2_c202502261200.average_masked.dat.tif"

# Shapefile with cell level (admin region level 4) boundaries 
FILE_SHP_ADM4 = DIR_DATA / "geospatial/pop-demo-infra/adm4-pop-buildings.shp"

# Shapefile with national boundaries (admin region level 0)
FILE_SHP_ADM0 = DIR_DATA / "geospatial/adm-bounds/geoBoundaries-RWA-ADM0-all/geoBoundaries-RWA-ADM0.shp"

## Zonal Statistics Analysis

### Clip Global Raster to Rwanda 
To make the raster file smaller and more manageable, we will clip (cut) it to the boundary of Rwanda. This step extracts only the portion of the raster that falls within Rwanda’s borders, reducing file size and improving processing efficiency.

In [11]:
def clip_raster_to_boundary(input_raster_path, boundary_gdf, output_path):
    """
    Clip a global raster to a country boundary
    
    Parameters:
    -----------
    input_raster_path : str
        Path to input raster file
    boundary_gdf : geopandas.GeoDataFrame
        Boundary for clipping
    output_path : str
        Path for output clipped raster
    
    Returns:
    --------
    str : Path to clipped raster
    """
    print(f"Clipping global raster to Rwanda boundary...")
    
    with rasterio.open(input_raster_path) as src:
        print(f"Original raster info:")
        print(f"  - Shape: {src.width} x {src.height}")
        print(f"  - CRS: {src.crs}")
        print(f"  - Bounds: {src.bounds}")
        print(f"  - Resolution: {src.res}")
        
        # Reproject boundary to match raster CRS if needed
        if boundary_gdf.crs != src.crs:
            print(f"Reprojecting boundary from {boundary_gdf.crs} to {src.crs}")
            boundary_gdf = boundary_gdf.to_crs(src.crs)
        
        # Clip raster
        clipped_data, clipped_transform = mask(
            src, 
            boundary_gdf.geometry, 
            crop=True,
            nodata=src.nodata
        )
        
        # Update metadata
        clipped_meta = src.meta.copy()
        clipped_meta.update({
            "driver": "GTiff",
            "height": clipped_data.shape[1],
            "width": clipped_data.shape[2],
            "transform": clipped_transform
        })
        
        # Create output directory if needed
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Write clipped raster
        with rasterio.open(output_path, "w", **clipped_meta) as dest:
            dest.write(clipped_data)
        
        print(f"✓ Clipped raster saved to: {output_path}")
        print(f"Clipped raster info:")
        print(f"  - Shape: {clipped_data.shape[2]} x {clipped_data.shape[1]}")
        print(f"  - Data range: {np.nanmin(clipped_data)} to {np.nanmax(clipped_data)}")
        
    return output_path

In [12]:
# =============================================
# CLIP THE GLOBAL RASTER TO RWANDA
# ===============================================

# Specify output clipped raster file
clipped_raster_path = DIR_DATA / "ntl/rwanda_2024_ntl.tif"

# Load level 0 boundaries in a GeoDataframe
gdf_adm0 = gp.read_file(FILE_SHP_ADM0)


# Call the clip raster function which utilises rasterio package
rwanda_raster_path = clip_raster_to_boundary(
    FILE_NTL, 
    gdf_adm0, 
    clipped_raster_path
)


DataSourceError: c:\Users\malio\Desktop\nightlight-analysis-Obed\nightlight-analysis\data\geospatial\adm-bounds\geoBoundaries-RWA-ADM0-all\geoBoundaries-RWA-ADM0.shp: No such file or directory

### Visualize Clipped Raster

In [13]:
# =========================================
# LOAD AND VISUALIZE THE CLIPPED RASTER
# =========================================

# Load cell boundaries
gdf_adm4 = gp.read_file(FILE_SHP_ADM4)

with rasterio.open(rwanda_raster_path) as src:
    rwanda_nightlight = src.read(1)
    rwanda_extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

# Create visualization
plt.figure(figsize=(15, 10))

# Plot 1: Clipped raster
plt.subplot(2, 2, 1)
plt.imshow(rwanda_nightlight, extent=rwanda_extent, cmap='viridis', alpha=0.8)
plt.colorbar(label='Nightlight Intensity', shrink=0.7)
plt.title('Rwanda Nightlight Data (Clipped)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Plot 2: Raster with district boundaries
plt.subplot(2, 2, 2)
plt.imshow(rwanda_nightlight, extent=rwanda_extent, cmap='viridis', alpha=0.8)
rwanda_districts_plot = gdf_adm4.to_crs(gdf_adm0.crs) if gdf_adm4.crs != gdf_adm0.crs else gdf_adm4
rwanda_districts_plot.boundary.plot(ax=plt.gca(), color='white', linewidth=0.5)
plt.colorbar(label='Nightlight Intensity', shrink=0.7)
plt.title('Rwanda Nightlight with District Boundaries')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

DataSourceError: c:\Users\malio\Desktop\nightlight-analysis-Obed\nightlight-analysis\data\geospatial\pop-demo-infra\adm4-pop-buildings.shp: No such file or directory

### Generate Cell Level Zonal Statistics

In [ ]:
# ===================================================
# GENERATING ZONAL STATISTICS WITH A CUSTOM SCRIPT
# ===================================================

# Create ZonalStatistics object using the clipped Rwanda data with
# the following args
# 1. Path of clipped night lights raster/image: rwanda_raster_path
# 2. Path of administrative boundaries shapefile: FILE_SHP_ADM4
# 3. Administrative level: 4

admin_level = "cell_id"

zs = ZonalStatistics(rwanda_raster_path, FILE_SHP_ADM4, admin_level)


In [ ]:
# Load the data (this will print information about the datasets)
zs.load_data()


In [ ]:
# =======================================
# DEFINE WHICH STATISTICS TO CALCULATE
# =======================================
stats_to_calculate = ['mean', 'median', 'count', "sum"]

print(f"Calculating statistics: {stats_to_calculate}")
print("This may take a few minutes depending on data size...")

# Calculate statistics
df_stats = zs.calculate_statistics(stats_to_calculate)

# Display results
print(f"\nAnalysis complete! Results shape: {df_stats.shape}")
display(df_stats.head(10))

In [ ]:
# ==================================================
# SAVE RESULTS
# ==================================================

# Ensure we grab region names from original shapefile
cols2keep = ['cell_id', 'province_n', 'district_n', 'sector_nam', 'cell_name']
df_stats2 = df_stats.merge(gdf_adm4[cols2keep], left_on="admin_name", right_on='cell_id', how='left')

# Save results with the following columns
cols2keep = ['province_n', 'district_n', 'sector_nam',  'cell_name', 'mean', 'median', 'count',
       'sum']

df_stats2[cols2keep].to_csv(DIR_DATA / "ntl/rw-adm4-ntl-stats.csv", index=False)

In [ ]:
## Report on Cells with Largest Night Light Values
largest_cells = df_stats2.nlargest(10, 'mean')
print("Cells with the largest night light values:")
display(largest_cells)